 # Nike Sales Analysis 

 ## 1. Project Overview
 This notebook performs **analysis** of Nike sales data.  
 **Goals:**
 - Clean and validate a real‑world‑like dataset with 50%+ missing values.
 - Apply rigorous imputation, outlier treatment

 **Key challenges tackled:**
 - Mixed date formats, impossible discounts, negative sales (returns).
 - 66% missing discount rates, 50% missing MRP/Units_Sold.
 - 2334 zero‑revenue records – I flag them and treat them as cancelled/inventory.
 

## 2. Environment Setup

In [31]:
import pandas as pd
import numpy as np
import os

In [32]:
# Load dataset (relative path – adjust if necessary)
df = pd.read_csv(r'data\Nike_Sales_Uncleaned.csv')
print(f"✅ Dataset loaded. Initial shape: {df.shape}")

✅ Dataset loaded. Initial shape: (2500, 13)


## 3. Initial Data Inspection
I first examine the structure, data types, and a sample of rows.

In [33]:
print("\nSample of first 5 rows:")
df.head()


Sample of first 5 rows:


,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
0,2000,Kids,Training,SuperRep Go,M,NaN,NaN,0.47,0.0,2024-03-09,Online,bengaluru,-770.45
1,2001,Women,Soccer,Tiempo Legend,M,3.0,4957.93,NaN,0.0,2024-07-09,Retail,Hyd,-112.53
2,2002,Women,Soccer,Premier III,M,4.0,NaN,NaN,0.0,NaN,Retail,Mumbai,3337.34
3,2003,Kids,Lifestyle,Blazer Mid,L,NaN,9673.57,NaN,0.0,04-10-2024,Online,Pune,3376.85
4,2004,Kids,Running,React Infinity,XL,NaN,NaN,NaN,0.0,2024/09/12,Retail,Delhi,187.89


In [34]:
print("\nSample of last 5 rows:")
df.tail()


Sample of last 5 rows:


,Order_ID,Gender_Category,Product_Line,Product_Name,Size,Units_Sold,MRP,Discount_Applied,Revenue,Order_Date,Sales_Channel,Region,Profit
2495,4495,Kids,Basketball,Kyrie Flytrap,XL,3.0,NaN,NaN,0.0,2025-05-14,Online,Pune,2.97
2496,4496,Men,Basketball,Kyrie Flytrap,L,-1.0,NaN,NaN,0.0,NaN,Online,Hyd,-487.36
2497,4497,Men,Soccer,Tiempo Legend,7,NaN,6647.60,0.52,0.0,02-07-2025,Retail,Bangalore,-918.14
2498,4498,Women,Training,ZoomX Invincible,NaN,4.0,5358.70,NaN,0.0,11-12-2024,Online,Mumbai,3352.29
2499,4499,Women,Running,Air Zoom,M,NaN,5550.99,NaN,0.0,23-11-2024,Retail,Mumbai,2453.57


In [35]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2500 entries, 0 to 2499
Data columns (total 13 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Order_ID          2500 non-null   int64  
 1   Gender_Category   2500 non-null   object 
 2   Product_Line      2500 non-null   object 
 3   Product_Name      2500 non-null   object 
 4   Size              1990 non-null   object 
 5   Units_Sold        1265 non-null   float64
 6   MRP               1246 non-null   float64
 7   Discount_Applied  832 non-null    float64
 8   Revenue           2500 non-null   float64
 9   Order_Date        1884 non-null   object 
 10  Sales_Channel     2500 non-null   object 
 11  Region            2500 non-null   object 
 12  Profit            2500 non-null   float64
dtypes: float64(5), int64(1), object(7)
memory usage: 254.0+ KB


## 4. Data Quality Assessment

This step identifies all quality issues **before** any modification.


In [36]:
# 1 Missing values
missing = df.isnull().sum()
missing_pct = (missing / len(df) * 100).round(1)
missing_df = pd.DataFrame({
    'Missing_Count': missing[missing > 0],
    'Percentage': missing_pct[missing > 0]
}).sort_values('Percentage', ascending=False)
print("\nMISSING VALUES:\n", missing_df)


MISSING VALUES:
                   Missing_Count  Percentage
Discount_Applied           1668        66.7
MRP                        1254        50.2
Units_Sold                 1235        49.4
Order_Date                  616        24.6
Size                        510        20.4


In [37]:
# 2 Duplicates
duplicates = df.duplicated().sum()
print(f"\nDUPLICATES: {duplicates} duplicate rows")


DUPLICATES: 0 duplicate rows


In [38]:
# 3 Data anomalies
negative_units = (df['Units_Sold'] < 0).sum()
zero_units = (df['Units_Sold'] == 0).sum()
zero_revenue = (df['Revenue'] == 0).sum()
negative_profit = (df['Profit'] < 0).sum()
print("\nDATA ANOMALIES:")
print(f"   Negative Units_Sold (returns): {negative_units}")
print(f"   Zero Units_Sold: {zero_units}")
print(f"   Zero Revenue records: {zero_revenue}")
print(f"   Negative Profit records: {negative_profit}")


DATA ANOMALIES:
   Negative Units_Sold (returns): 205
   Zero Units_Sold: 224
   Zero Revenue records: 2334
   Negative Profit records: 578


In [39]:
# 4 Discount issues
invalid_discounts = (df['Discount_Applied'] > 1).sum()
print("\nDISCOUNT ANALYSIS:")
print(f"   Discounts > 100% (likely data entry errors): {invalid_discounts}")
if invalid_discounts > 0:
    print(f"   Sample invalid values: {df[df['Discount_Applied'] > 1]['Discount_Applied'].head().tolist()}")


DISCOUNT ANALYSIS:
   Discounts > 100% (likely data entry errors): 180
   Sample invalid values: [1.17, 1.05, 1.07, 1.01, 1.04]


In [40]:
# 5 Date formats
sample_dates = df['Order_Date'].head(5).tolist()
print("\nDATE FORMAT:")
print(f"   Sample dates: {sample_dates}")
print("   ⚠️  Mixed formats detected – will be standardized.")



DATE FORMAT:
   Sample dates: ['2024-03-09', '2024-07-09', nan, '04-10-2024', '2024/09/12']
   ⚠️  Mixed formats detected – will be standardized.


In [41]:
# 6 Categorical summary
print('\nUnique values in categorical columns:')
print(f"- Gender_Category: {df['Gender_Category'].unique()}")
print(f"- Product_Line: {df['Product_Line'].unique()}")
print(f"- Sales_Channel: {df['Sales_Channel'].unique()}")
print(f"- Region: {df['Region'].unique()}")
print(f"- Size: {df['Size'].unique()}")



Unique values in categorical columns:
- Gender_Category: ['Kids' 'Women' 'Men']
- Product_Line: ['Training' 'Soccer' 'Lifestyle' 'Running' 'Basketball']
- Sales_Channel: ['Online' 'Retail']
- Region: ['bengaluru' 'Hyd' 'Mumbai' 'Pune' 'Delhi' 'Bangalore' 'Hyderabad'
 'hyderbad' 'Kolkata']
- Size: ['M' 'L' 'XL' '11' nan '9' '6' '12' '7' '10' '8']


In [42]:
# 7 Numeric summary
print("\nNUMERIC COLUMNS SUMMARY:")
print(df[['Units_Sold', 'MRP', 'Discount_Applied', 'Revenue', 'Profit']].describe())



NUMERIC COLUMNS SUMMARY:
        Units_Sold          MRP  Discount_Applied       Revenue       Profit
count  1265.000000  1246.000000        832.000000   2500.000000  2500.000000
mean      1.482213  6039.863395          0.631022    274.873716  1376.012848
std       1.696947  2315.746826          0.365500   2023.773550  1478.671013
min      -1.000000  2006.630000          0.000000  -7561.590000 -1199.450000
25%       0.000000  4038.902500          0.320000      0.000000    93.532500
50%       2.000000  6110.030000          0.615000      0.000000  1371.385000
75%       3.000000  8022.187500          0.962500      0.000000  2660.645000
max       4.000000  9996.220000          1.250000  37169.350000  3999.210000


## 5. Data Cleaning

I fix structural errors **before** any imputation to avoid propagating garbage.


In [43]:
# --- Configurable parameters ---
MAX_ALLOWED_DISCOUNT = 1.0
DISCOUNT_OFFSET = 1.0
ZERO_REVENUE_THRESHOLD = 0.01

# Start with a clean copy
df_clean = df.copy()

In [44]:
# 5.1 Remove returns (negative sales)
initial_len = len(df_clean)
df_clean = df_clean[df_clean['Units_Sold'].fillna(0) >= 0].copy()
print(f"🔧 Removed {initial_len - len(df_clean)} return records")

🔧 Removed 205 return records


In [45]:
# 5.2 Fix impossible discounts ( > 100% )
mask_bad_discount = df_clean['Discount_Applied'] > MAX_ALLOWED_DISCOUNT
if mask_bad_discount.sum() > 0:
    df_clean.loc[mask_bad_discount, 'Discount_Applied'] -= DISCOUNT_OFFSET
    print(f"🔧 Corrected {mask_bad_discount.sum()} discount values (subtracted {DISCOUNT_OFFSET})")

🔧 Corrected 169 discount values (subtracted 1.0)


In [46]:
# 5.3 Standardize dates
# 'mixed' handles different formats; dayfirst=False interprets ambiguous dates as month‑first.
df_clean['Order_Date'] = pd.to_datetime(df_clean['Order_Date'], format='mixed', dayfirst=False, errors='coerce')
unparseable_dates = df_clean['Order_Date'].isna().sum()
print(f"🔧 Dates standardized. {unparseable_dates} unparseable dates set to NaT.")

🔧 Dates standardized. 569 unparseable dates set to NaT.


In [47]:
# 5.4 Clean region names
region_corrections = {
    'Hyd': 'Hyderabad',
    'hyderbad': 'Hyderabad',
    'Hyderbad': 'Hyderabad',
    'bengaluru': 'Bangalore'
}
df_clean['Region'] = df_clean['Region'].replace(region_corrections)
print(f"🔧 Region names standardized. Unique regions: {df_clean['Region'].unique()}")


🔧 Region names standardized. Unique regions: ['Bangalore' 'Hyderabad' 'Mumbai' 'Pune' 'Delhi' 'Kolkata']


In [48]:
# Define the mapping for categorical sizes to numeric equivalents
size_mapping = {
    'M': 8,
    'L': 10,
    'XL': 12
}

df_clean['Size'] = df_clean['Size'].replace(size_mapping)
df_clean['Size'] = pd.to_numeric(df_clean['Size'], errors='coerce')

print(f"🔧 Done convert categorical sizes to numeric  {df_clean['Size'].unique()}")



🔧 Done convert categorical sizes to numeric  [ 8. 10. 12. nan 11.  9.  6.  7.]


In [49]:
print(f"\n✅ Rows after structural cleaning: {len(df_clean)}")


✅ Rows after structural cleaning: 2295


## 6. Advanced Imputation Strategy

I used a **hybrid approach**:

In [50]:
# 6.1 Mathematical recovery
print("\n📐 Step 1: Mathematical Recovery")

# Recover Units_Sold from Revenue / MRP
mask_recover_units = (df_clean['Units_Sold'].isna() &
                      df_clean['Revenue'].notna() &
                      df_clean['MRP'].notna() &
                      (df_clean['MRP'] > 0))
df_clean.loc[mask_recover_units, 'Units_Sold'] = df_clean.loc[mask_recover_units, 'Revenue'] / df_clean.loc[mask_recover_units, 'MRP']
print(f"   ✓ Recovered {mask_recover_units.sum()} Units_Sold values")

# Recover MRP from Revenue / Units_Sold
mask_recover_mrp = (df_clean['MRP'].isna() &
                    df_clean['Revenue'].notna() &
                    df_clean['Units_Sold'].notna() &
                    (df_clean['Units_Sold'] > 0))
df_clean.loc[mask_recover_mrp, 'MRP'] = df_clean.loc[mask_recover_mrp, 'Revenue'] / df_clean.loc[mask_recover_mrp, 'Units_Sold']
print(f"   ✓ Recovered {mask_recover_mrp.sum()} MRP values")


📐 Step 1: Mathematical Recovery
   ✓ Recovered 613 Units_Sold values
   ✓ Recovered 416 MRP values


In [51]:
# 6.2 Statistical imputation for MRP
print("\n📊 Step 2: Statistical Imputation")

# Product‑specific median
df_clean['MRP'] = df_clean.groupby('Product_Name')['MRP'].transform(
    lambda x: x.fillna(x.median())
)
# Product‑line median as fallback
df_clean['MRP'] = df_clean.groupby('Product_Line')['MRP'].transform(
    lambda x: x.fillna(x.median())
)
# Global median as final fallback
global_mrp_median = df_clean['MRP'].median()
df_clean['MRP'] = df_clean['MRP'].fillna(global_mrp_median)
print(f"   ✓ Imputed remaining MRP values (product median → product line median → global median)")


📊 Step 2: Statistical Imputation
   ✓ Imputed remaining MRP values (product median → product line median → global median)


In [52]:
# 6.3 Baseline imputation for Units_Sold with flagging
print("\n⚠️  Step 3: Baseline Units_Sold Imputation")
print(f"   • Missing Units_Sold with Revenue > {ZERO_REVENUE_THRESHOLD} → 1 (conservative baseline)")
print(f"   • All other missing Units_Sold → 0 (zero sales)")

df_clean['Units_Imputed_Baseline'] = False
mask_impute_units = df_clean['Units_Sold'].isna() & (df_clean['Revenue'] > ZERO_REVENUE_THRESHOLD)
df_clean.loc[mask_impute_units, 'Units_Sold'] = 1
df_clean.loc[mask_impute_units, 'Units_Imputed_Baseline'] = True

# Remaining missing Units_Sold (zero‑revenue rows) set to 0
df_clean['Units_Sold'] = df_clean['Units_Sold'].fillna(0)
print(f"   ✓ Imputed {mask_impute_units.sum()} Units_Sold with 1")
print(f"   ✓ Set {df_clean['Units_Sold'].isna().sum()} remaining missing Units_Sold to 0 (now all filled)")


⚠️  Step 3: Baseline Units_Sold Imputation
   • Missing Units_Sold with Revenue > 0.01 → 1 (conservative baseline)
   • All other missing Units_Sold → 0 (zero sales)
   ✓ Imputed 0 Units_Sold with 1
   ✓ Set 0 remaining missing Units_Sold to 0 (now all filled)


In [53]:
# 6.4 Impute Discount_Applied (0% = no discount)
discount_before = df_clean['Discount_Applied'].isna().sum()
df_clean['Discount_Applied'] = df_clean['Discount_Applied'].fillna(0)
print(f"   ✓ Imputed {discount_before} missing Discount_Applied with 0")

   ✓ Imputed 1528 missing Discount_Applied with 0


In [54]:
# 6.5 Impute Size using mode per product
size_before = df_clean['Size'].isna().sum()
df_clean['Size'] = df_clean.groupby('Product_Name')['Size'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
)
# Fallback: Gender + Product Line mode
df_clean['Size'] = df_clean.groupby(['Gender_Category', 'Product_Line'])['Size'].transform(
    lambda x: x.fillna(x.mode()[0] if not x.mode().empty else np.nan)
)
df_clean['Size'] = df_clean['Size'].fillna(df_clean['Size'].mode()[0])
print(f"   ✓ Imputed {size_before} Size values (Product Mode → Category Mode → Global Mode)")

   ✓ Imputed 469 Size values (Product Mode → Category Mode → Global Mode)


In [55]:
# 6.6 Revenue recalculation for consistency
print("\n🔒 Step 4: Revenue Consistency Lock")
df_clean['Revenue'] = (df_clean['Units_Sold'] * df_clean['MRP']) * (1 - df_clean['Discount_Applied'])
print(f"   ✓ Revenue recalculated as (Units_Sold × MRP) * (1 - Discount) (100% consistent)")


🔒 Step 4: Revenue Consistency Lock
   ✓ Revenue recalculated as (Units_Sold × MRP) * (1 - Discount) (100% consistent)


In [56]:
# 6.7 Flag zero‑sales records
df_clean['Is_Zero_Sales'] = (df_clean['Units_Sold'] == 0) & (df_clean['MRP'] > 0)
print(f"\n⚠️  Flagged {df_clean['Is_Zero_Sales'].sum()} records as 'Is_Zero_Sales' (Units=0 but MRP>0)")
print("   These may represent inventory records, cancelled orders, or data entry errors.")


⚠️  Flagged 1459 records as 'Is_Zero_Sales' (Units=0 but MRP>0)
   These may represent inventory records, cancelled orders, or data entry errors.


In [57]:
print("\n📅 Step 6.8: Handling Missing Order_Date with Flag")

# 1. Create a Flag Column for missing Order_Date
df_clean['Date_Missing_Flag'] = df_clean['Order_Date'].isna()
missing_count_before = df_clean['Date_Missing_Flag'].sum()

print(f"   ✓ Created 'Date_Missing_Flag' for {missing_count_before} records with missing Order_Date.")
print("   • Order_Date column will retain NaT values for missing dates.")


📅 Step 6.8: Handling Missing Order_Date with Flag
   ✓ Created 'Date_Missing_Flag' for 569 records with missing Order_Date.
   • Order_Date column will retain NaT values for missing dates.


In [58]:
print("\n✅ Imputation Complete!")
print(f"Remaining missing values:\n{df_clean.isnull().sum()[df_clean.isnull().sum() > 0]}")


✅ Imputation Complete!
Remaining missing values:
Order_Date    569
dtype: int64


## 7. Outlier Handling (Winsorization)

 I apply capping only to **positive revenue** values to avoid distortion from the large zero‑revenue group.  
 The IQR is calculated exclusively on `Revenue > 0`, and values above the upper bound are capped.


In [59]:
# Isolate positive revenues for IQR calculation
positive_rev = df_clean.loc[df_clean['Revenue'] > 0, 'Revenue']
if len(positive_rev) > 0:
    Q1 = positive_rev.quantile(0.25)
    Q3 = positive_rev.quantile(0.75)
    IQR = Q3 - Q1
    upper_bound = Q3 + 1.5 * IQR
    print(f"Positive revenue Q1:  ${Q1:,.2f}")
    print(f"Positive revenue Q3:  ${Q3:,.2f}")
    print(f"IQR:                 ${IQR:,.2f}")
else:
    # Fallback if no positive revenues (should not happen)
    upper_bound = df_clean['Revenue'].quantile(0.99)
print(f"Upper bound:         ${upper_bound:,.2f}")

# Cap revenue at the upper bound
df_clean['Revenue_Capped'] = np.where(
    df_clean['Revenue'] > upper_bound,
    upper_bound,
    df_clean['Revenue']
)

outliers_capped = (df_clean['Revenue'] > upper_bound).sum()
print(f"\nOutliers capped: {outliers_capped} rows")
print(f"Max Revenue before capping: ${df_clean['Revenue'].max():,.2f}")
print(f"Max Revenue after capping:  ${df_clean['Revenue_Capped'].max():,.2f}")
print("\n✅ Outlier capping applied (original Revenue column preserved)")
print("="*70)


Positive revenue Q1:  $5,601.38
Positive revenue Q3:  $18,030.43
IQR:                 $12,429.06
Upper bound:         $36,674.02

Outliers capped: 7 rows
Max Revenue before capping: $39,984.88
Max Revenue after capping:  $36,674.02

✅ Outlier capping applied (original Revenue column preserved)


In [60]:
output_dir = 'Outputs'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(output_dir, 'Nike_Sales_Final_Cleaned.csv')
df_clean.to_csv(output_file, index=False)
print(f"✅ Exported successfully to: {output_file}")

✅ Exported successfully to: Outputs\Nike_Sales_Final_Cleaned.csv
